# Explore and cluster the neighborhoods in Toronto

### Import necessary libraries

In [11]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors


### Dataset of task2

In [12]:
df3=pd.read_csv('Capstone_Task2.csv')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df4=df3[df3['Borough'].str.contains('Toronto')]
df5=df4.reset_index(drop=True)
df5.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
df5.shape

(39, 5)

In [16]:
df5['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [17]:
df5['Label']=df5['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df5.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Label
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


In [18]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### k-means clustering

In [19]:
kclusters=len(df5.Label.unique())

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Label']):
    label = folium.Popup(str(df5['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

In [20]:
map_toronto